# Project 2 Part 3B
**Transform and Load (core)**


*Christina Brockway*

### Task
- Take the cleaned data and add stakeholder data to SQL database
- Genres
    -  use title basics from part 1
    -  extract and normalize column
- New Table from API calls
    -  Tmdb_data table
        -  imdb_id
        -  revenue
        -  budget
        -  certification

### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json


In [ ]:
### Load data

In [ ]:
df = pd.read_csv("/MovieData/tmdb_final_results_combined.csv.gz")

## Clean Files

### Generes

In [ ]:
# screate a new column with a list of genres
df['genres_split']= df['genre'].str.split(",")
df

In [ ]:
#Seperate list of genres
exploded_genres = df.explode('genres_sp
df.heat(2)

In [ ]:
df- df.drop (columns=